In [1]:
!pip install tensorflow==2.2.0-rc2
!pip install tensorflow-addons

     |████████████████████████████████| 516.2 MB 2.5 kB/s 
     |████████████████████████████████| 454 kB 39.1 MB/s 
     |████████████████████████████████| 3.0 MB 44.4 MB/s 
     |████████████████████████████████| 777 kB 41.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip inst

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [3]:
import copy
import math
import os
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import tensorflow.keras.layers as L
from tensorflow.keras import Model
from sklearn.metrics import f1_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ModelCheckpoint
import math
# from tqdm.notebook import tqdm
import gc
import os
from sklearn.metrics import recall_score, f1_score, precision_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from sklearn.model_selection import KFold, GroupKFold
from tensorflow.python.keras.engine import training
import collections
from scipy import signal

LEN_ = 4000
n_epoch = 300
batch_size = 64
folds = 5
learning_rate = 0.001
SEED_ = 987654321
FOLD_ = 0
n_classes = 11

lr_schedule = LearningRateScheduler(
    PolynomialDecay(
      initial_learning_rate = learning_rate,
      decay_steps = n_epoch,
      power = 0.5,
      end_learning_rate = 0.0
    )
)



In [4]:
class FBetaScore(tf.keras.metrics.Metric):
    """Computes F-Beta score.
    It is the weighted harmonic mean of precision
    and recall. Output range is [0, 1]. Works for
    both multi-class and multi-label classification.
    F-Beta = (1 + beta^2) * (prec * recall) / ((beta^2 * prec) + recall)
    Args:
        num_classes: Number of unique classes in the dataset.
        average: Type of averaging to be performed on data.
            Acceptable values are `None`, `micro`, `macro` and
            `weighted`. Default value is None.
        beta: Determines the weight of precision and recall
            in harmonic mean. Determines the weight given to the
            precision and recall. Default value is 1.
        threshold: Elements of `y_pred` greater than threshold are
            converted to be 1, and the rest 0. If threshold is
            None, the argmax is converted to 1, and the rest 0.
    Returns:
        F-Beta Score: float
    Raises:
        ValueError: If the `average` has values other than
        [None, micro, macro, weighted].
        ValueError: If the `beta` value is less than or equal
        to 0.
    `average` parameter behavior:
        None: Scores for each class are returned
        micro: True positivies, false positives and
            false negatives are computed globally.
        macro: True positivies, false positives and
            false negatives are computed for each class
            and their unweighted mean is returned.
        weighted: Metrics are computed for each class
            and returns the mean weighted by the
            number of true instances in each class.
    """

    def __init__(self,
                 num_classes,
                 shape = [LEN_, n_classes],
                 average=None,
                 beta=1.0,
                 threshold=None,
                 name='fbeta_score',
                 dtype=tf.float32):
        super(FBetaScore, self).__init__(name=name)

        if average not in (None, 'micro', 'macro', 'weighted'):
            raise ValueError("Unknown average type. Acceptable values "
                             "are: [None, micro, macro, weighted]")

        if not isinstance(beta, float):
            raise TypeError("The value of beta should be a python float")

        if beta <= 0.0:
            raise ValueError("beta value should be greater than zero")

        if threshold is not None:
            if not isinstance(threshold, float):
                raise TypeError(
                    "The value of threshold should be a python float")
            if threshold > 1.0 or threshold <= 0.0:
                raise ValueError("threshold should be between 0 and 1")

        self.num_classes = num_classes
        self.average = average
        self.beta = beta
        self.threshold = threshold
        self.axis = None
        self.init_shape = shape
        
        if self.average != 'micro':
            self.axis = 0
            # self.init_shape = [shape]

        def _zero_wt_init(name):
            return self.add_weight(
                name,
                shape=self.init_shape,
                initializer='zeros',
                dtype=self.dtype)

        self.true_positives = _zero_wt_init('true_positives')
        self.false_positives = _zero_wt_init('false_positives')
        self.false_negatives = _zero_wt_init('false_negatives')
        self.weights_intermediate = _zero_wt_init('weights_intermediate')

    # TODO: Add sample_weight support, currently it is
    # ignored during calculations.
    def update_state(self, y_true, y_pred, sample_weight=None):
        if self.threshold is None:
            threshold = tf.reduce_max(y_pred, axis=-1, keepdims=True)
            # make sure [0, 0, 0] doesn't become [1, 1, 1]
            # Use abs(x) > eps, instead of x != 0 to check for zero
            y_pred = tf.logical_and(y_pred >= threshold,
                                    tf.abs(y_pred) > 1e-12)
        else:
            y_pred = y_pred > self.threshold

        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)

        def _count_non_zero(val):
            non_zeros = tf.math.count_nonzero(val, axis=self.axis)
            return tf.cast(non_zeros, self.dtype)

        self.true_positives.assign_add(_count_non_zero(y_pred * y_true))
        self.false_positives.assign_add(_count_non_zero(y_pred * (y_true - 1)))
        self.false_negatives.assign_add(_count_non_zero((y_pred - 1) * y_true))
        self.weights_intermediate.assign_add(_count_non_zero(y_true))

    def result(self):
        precision = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_positives)
        recall = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_negatives)

        mul_value = precision * recall
        add_value = (tf.math.square(self.beta) * precision) + recall
        mean = (tf.math.divide_no_nan(mul_value, add_value))
        f1_score = mean * (1 + tf.math.square(self.beta))

        if self.average == 'weighted':
            weights = tf.math.divide_no_nan(
                self.weights_intermediate,
                tf.reduce_sum(self.weights_intermediate))
            f1_score = tf.reduce_sum(f1_score * weights)

        elif self.average is not None:  # [micro, macro]
            f1_score = tf.reduce_mean(f1_score)

        return f1_score

    def get_config(self):
        """Returns the serializable config of the metric."""

        config = {
            "num_classes": self.num_classes,
            "average": self.average,
            "beta": self.beta,
        }

        if self.threshold is not None:
            config["threshold"] = self.threshold

        base_config = super(FBetaScore, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def reset_states(self):
        self.true_positives.assign(tf.zeros(self.init_shape, self.dtype))
        self.false_positives.assign(tf.zeros(self.init_shape, self.dtype))
        self.false_negatives.assign(tf.zeros(self.init_shape, self.dtype))
        self.weights_intermediate.assign(tf.zeros(self.init_shape, self.dtype))


In [5]:
class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

get_custom_objects().update({'Mish': Mish(mish)})

class macroF1(Callback):
    def __init__(self, model, inputs, targets, step = 10):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)
        self.step = step

    def on_train_begin(self, logs={}):    
            # Initialization code    
            self.epochs = 0    

    def on_epoch_end(self, epoch, logs):
        self.epochs += 1   
        if self.epochs % self.step == 0:  
          pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
          f1_val = f1_score(self.targets, pred, average="macro")
          print("Step: {}  --  val_f1_macro_score: {}".format(self.epochs, f1_val))

In [6]:
def expand_dims(x, channels_axis):
    return x[:, None, :]

def ChannelSE(step, reduction = 16):
    """
    Squeeze and Excitation block, reimplementation inspired by
        https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/senet.py
    Args:
        reduction: channels squeeze factor
    """

    channels_axis = 2

    def layer(input_tensor):

        channels = tf.keras.backend.int_shape(input_tensor)[channels_axis]

        # get number of channels/filters
        x = input_tensor

        # squeeze and excitation block in PyTorch style with
        x = layers.GlobalAveragePooling1D(name = f'att_global_pool_{step}')(x)
        x = layers.Lambda(expand_dims, arguments={'channels_axis': channels_axis})(x)

        x = layers.Conv1D(channels // reduction, 1, kernel_initializer='he_uniform', name = f'att_conv_1_{step}')(x)
        x = layers.Activation('Mish')(x)
        x = layers.Conv1D(channels, 1, kernel_initializer='he_uniform', name = f'att_conv_2_{step}')(x)
        x = layers.Activation('sigmoid')(x)

        # apply attention
        x = layers.Multiply()([input_tensor, x])

        return x

    return layer



In [7]:
def bilstm(x, n_units, name):
    x = L.Bidirectional(L.GRU(n_units, return_sequences=True), name = 'step' + name + str(n_units))(x)
    return(x)

def bi_block(x, n_units):
    y = bilstm(x, n_units, name = '_first_')
    y_1 = bilstm(y, n_units, name = '_second_')
    y_2 = bilstm(y_1, n_units, name = '_third_')
    z = ChannelSE(step = n_units)(y_2)
    x = layers.Add(name = f'step_add_{n_units}')([y, z])
    return(x)


def bilstm_model(shape_ = (LEN_, 3), n_units = 128):
    input_layer = L.Input(shape = shape_, name = 'input_layer')
    bi_32 = bi_block(input_layer, n_units = n_units//4)
    bi_64 = bi_block(bi_32, n_units = n_units//2)
    
    bi_128 = bi_block(bi_64, n_units = n_units)
    x = L.Dropout(0.1, noise_shape = (None, 1, 256))(bi_128)
    
    x = L.TimeDistributed(L.Dense(n_units, activation = 'Mish'), name = 'first_distributed')(x)
    x = L.Dropout(0.1, noise_shape = (None, 1, 128))(x)
    
    x = L.TimeDistributed(L.Dense(n_units//2, activation = 'Mish'), name = 'second_distributed')(x)
    x = L.TimeDistributed(L.Dense(n_classes, activation = 'softmax'), name = 'output')(x)
    
    model = Model(inputs=input_layer, outputs=x)    
    model.compile(Adam(learning_rate = learning_rate), loss = [tfa.losses.SigmoidFocalCrossEntropy()], 
          metrics=[FBetaScore(n_classes, average = "macro")])

    return model

def load_trained_model(weights_path, shape_):
    modelf = bilstm_model(shape_ = shape_)
    modelf.load_weights(weights_path)
    
    return(modelf)


In [8]:
path = '../input/kalman-add/'

group = np.load(path + 'group.npy', allow_pickle = True)
test_signal = np.load(path + 'test_signal.npy', allow_pickle = True)
train_signal = np.load(path + 'train_signal.npy', allow_pickle = True)
train_target = np.load(path + 'train_target.npy', allow_pickle = True)

In [9]:
out_ = np.arange(910, 956)
idx = np.arange(train_signal.shape[0])
bool_mask = np.isin(idx, out_)
train_signal = train_signal[~bool_mask]
group = group[~bool_mask]
train_target = train_target[~bool_mask]

In [10]:
model = bilstm_model(shape_ = (None, train_signal.shape[-1]))
print(f"Number of parameter used: {model.count_params()}")
del model
gc.collect()

Number of parameter used: 1088743


519

In [11]:
%%time
gc.collect()
kf = GroupKFold(n_splits = folds)
pred_oof = np.zeros((train_signal.shape[0], train_signal.shape[1], 11))

for fold_n, (train_index, valid_index) in enumerate(kf.split(train_signal, train_target, group)):
    if fold_n == FOLD_:
        print(f'BEGIN FOLD:     {fold_n} -------\n\n\n')

        model = bilstm_model(shape_ = (None, train_signal.shape[-1]))

        X_train, X_valid = train_signal[train_index,:,:], train_signal[valid_index,:,:]
        y_train, y_valid = train_target[train_index,:,:], train_target[valid_index,:,:]

        callBacks = [lr_schedule, macroF1(model, X_valid, y_valid)] 

        history = model.fit(
            X_train, y_train,
            epochs = n_epoch,
            batch_size = batch_size,
            callbacks = callBacks,
            verbose = 0,
            validation_data = (X_valid, y_valid)
        )
        pred_oof[valid_index, :, :] = model.predict(X_valid)
        pred = model.predict(test_signal)

        model.save_weights(f'model{str(fold_n)}.hdf5')
        print(f'\n\n\nENDED FOLD:     {fold_n} -------\n\n\n')

BEGIN FOLD:     0 -------



Step: 10  --  val_f1_macro_score: 0.9292070804975677
Step: 20  --  val_f1_macro_score: 0.937911724408703
Step: 30  --  val_f1_macro_score: 0.9386945446476321
Step: 40  --  val_f1_macro_score: 0.9378157273292149
Step: 50  --  val_f1_macro_score: 0.9395523264805367
Step: 60  --  val_f1_macro_score: 0.9398186274141735
Step: 70  --  val_f1_macro_score: 0.9396570733880562
Step: 80  --  val_f1_macro_score: 0.9393766490273987
Step: 90  --  val_f1_macro_score: 0.9392655372851181
Step: 100  --  val_f1_macro_score: 0.938383201819401
Step: 110  --  val_f1_macro_score: 0.9398789830677202
Step: 120  --  val_f1_macro_score: 0.93930111704607
Step: 130  --  val_f1_macro_score: 0.9407069996478228
Step: 140  --  val_f1_macro_score: 0.9400177306978055
Step: 150  --  val_f1_macro_score: 0.9404188899281176
Step: 160  --  val_f1_macro_score: 0.9403329957694964
Step: 170  --  val_f1_macro_score: 0.9399642707493037
Step: 180  --  val_f1_macro_score: 0.9406905478260509
Step: 190  -

In [12]:
%%time
np.save("pred" + str(FOLD_) + ".npy", pred, allow_pickle = True)
np.save("predoof" + str(FOLD_) + '.npy', pred_oof, allow_pickle = True)


CPU times: user 2.91 ms, sys: 421 ms, total: 424 ms
Wall time: 704 ms
